In [1]:
import blimpy

In [2]:
FILENAME = "data/blc41_guppi_59103_01372_PSR_B2021+51_0009.rawspec.0000.h5"

In [4]:
fb = blimpy.Waterfall(FILENAME)

In [6]:
fb.info()


--- File Info ---
DIMENSION_LABELS :   ['time' 'feed_id' 'frequency']
        az_start :                              0.0
       data_type :                                1
            fch1 :                2063.96484375 MHz
            foff :      -2.7939677238464355e-06 MHz
           ibeam :                               -1
      machine_id :                               20
          nbeams :                                1
           nbits :                               32
          nchans :                         67108864
            nifs :                                1
     rawdatafile : guppi_59103_01372_PSR_B2021+51_0009.0000.raw
     source_name :                     PSR_B2021+51
         src_dej :                      51:54:49.32
         src_raj :                      20:22:50.16
    telescope_id :                                6
           tsamp :               18.253611007999982
   tstart (ISOT) :          2020-09-11T00:22:52.000
    tstart (MJD) :               

In [8]:
fb.data.shape

(16, 1, 67108864)

In [9]:
fb.data[5][0][30]

13690.115